# Deriving Weqetal

The ETCBC does not mark weqetal verbs. We find them here.

In [66]:
from tf.app import use
import collections, random

In [39]:
A = use('bhsa', hoist=globals())

TF.load('tab', add=True)

TF app is up-to-date.
Using annotation/app-bhsa commit 7f353d587f4befb6efe1742831e28f301d2b3cea (=latest)
  in /Users/cody/text-fabric-data/__apps__/bhsa.
Using etcbc/bhsa/tf - c rv1.6 in /Users/cody/text-fabric-data
Using etcbc/phono/tf - c r1.2 in /Users/cody/text-fabric-data
Using etcbc/parallels/tf - c r1.2 in /Users/cody/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://github.com/annotation/app-bhsa" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Api/Fabric/" title="text-fabric-api">Text-Fabric API 7.4.6</a> <a target="_blank" href="https://annotation.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

  0.00s loading features ...
   |     0.14s T tab                  from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
  0.15s All additional features loaded - for details use loadLog()


## Develop

The algorithms below are applied in the duratives study.

In [40]:
def get_grandma(clause_atom, fam_tree):
    '''
    Recursively climbs up a qatal clause chain.
    Stops upon identifying either wayyiqtol
    or a yiqtol|impv grand(mother).
    '''
    this_verb = next((F.vt.v(w) for w in L.d(clause_atom) if F.pdp.v(w)=='verb'), '')
    mother = next((m for m in E.mother.f(clause_atom)), 0)
    mom_verb = next((F.vt.v(w) for w in L.d(mother) if F.pdp.v(w)=='verb'), '')    
    fam_tree.append(mother)
    
    if mom_verb in {'wayq', 'impf', 'impv'}:
        return mom_verb
    
    elif not mother:
        return this_verb
    
    else:
        return get_grandma(mother, fam_tree)

def filter_tense(verb_node, ancestors):
    '''
    Identifies and tags weqetal verbs.
    For all other, returns tense value.
    Uses the mother clause from the ETCBC and 
    looks for immediately preceding waw.
    '''
    if F.vt.v(verb_node) == 'perf' and F.lex.v(verb_node-1) == 'W':
        # check mother
        clause = L.u(verb_node, 'clause_atom')[0]
        qatal_ancestor = get_grandma(clause, ancestors)
        if qatal_ancestor in {'impf','impv'}:
            return ('weqt', ancestors)
        else:
            return (F.vt.v(verb_node), ancestors)
    return (F.vt.v(verb_node), ancestors)

In [41]:
testset = A.search('''

clause_atom
    word pdp=verb vt=perf

''')

  0.61s 21128 results


In [64]:
it = 0
weqtl = []

for clause, verb in testset:
    
    if it > 20:
        break
    
    ancestors = []
    tense, ancestors = filter_tense(verb, ancestors)
    
    if tense=='weqt':
        
        interval = sorted([clause, ancestors[-1]])
        kids = list(range(interval[0], interval[1]+1))

        weq_data = [f'{tense} {(str(clause))}']
        for kid in kids:
            indent = '.' * F.tab.v(kid)
            weq_data.append(f'{str(kid)} {T.text(kid)}{indent}')       
            
        weqtl.append(weq_data)

In [68]:
for i in range(0, 25):
    ex = random.choice(weqtl)
    print('\n'.join(ex))
    print('-'*20)
    print()

weqt 528114
528040 בְּזֹ֛את יָבֹ֥א אַהֲרֹ֖ן אֶל־הַקֹּ֑דֶשׁ ....
528041 בְּפַ֧ר בֶּן־בָּקָ֛ר לְחַטָּ֖את .......
528042 וְאַ֥יִל לְעֹלָֽה׃ ........
528043 כְּתֹֽנֶת־בַּ֨ד קֹ֜דֶשׁ יִלְבָּ֗שׁ ......
528044 וּמִֽכְנְסֵי־בַד֮ יִהְי֣וּ עַל־בְּשָׂרֹו֒ ..........
528045 וּבְאַבְנֵ֥ט בַּד֙ יַחְגֹּ֔ר ...........
528046 וּבְמִצְנֶ֥פֶת בַּ֖ד יִצְנֹ֑ף ...........
528047 בִּגְדֵי־קֹ֣דֶשׁ הֵ֔ם .........
528048 וְרָחַ֥ץ בַּמַּ֛יִם אֶת־בְּשָׂרֹ֖ו ........
528049 וּלְבֵשָֽׁם׃ .........
528050 וּמֵאֵ֗ת עֲדַת֙ בְּנֵ֣י יִשְׂרָאֵ֔ל יִקַּ֛ח שְׁנֵֽי־שְׂעִירֵ֥י עִזִּ֖ים לְחַטָּ֑את וְאַ֥יִל אֶחָ֖ד לְעֹלָֽה׃ .......
528051 וְהִקְרִ֧יב אַהֲרֹ֛ן אֶת־פַּ֥ר הַחַטָּ֖את .....
528052 אֲשֶׁר־לֹ֑ו .......
528053 וְכִפֶּ֥ר בַּעֲדֹ֖ו וּבְעַ֥ד בֵּיתֹֽו׃ ......
528054 וְלָקַ֖ח אֶת־שְׁנֵ֣י הַשְּׂעִירִ֑ם ......
528055 וְהֶעֱמִ֤יד אֹתָם֙ לִפְנֵ֣י יְהוָ֔ה פֶּ֖תַח אֹ֥הֶל מֹועֵֽד׃ ........
528056 וְנָתַ֧ן אַהֲרֹ֛ן עַל־שְׁנֵ֥י הַשְּׂעִירִ֖ם גֹּורָלֹ֑ות .......
528057 גֹּורָ֤ל אֶחָד֙ לַיהוָ֔ה ........
528058 וְגֹורָ֥ל אֶחָ֖ד לַעֲזָאז